In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 每个批次的大小
batch_size = 100
n_batch = mnist.train.num_examples//batch_size

# 给输入数据预留位置
# none表示第一个维度可以是任意的长度
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

# 创建无中间层的简单网络
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
Wx_plus_b = tf.matmul(x,W) + b

# 预测值
prediction = tf.nn.softmax(Wx_plus_b)

# 准备拟合
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

train = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

# 初始化变量
init = tf.global_variables_initializer()

# 定义求准确率的方法
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(51):                                                                                              # 总共训练21次
        for pic in range(n_batch):                                # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
            pic_data,pic_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
            sess.run(train,feed_dict={x:pic_data,y:pic_tag})        # 获取一个训练数据 这儿就训练一次   
        # 每一次大循环 看一下准确率
        if epoch % 10 == 0:
            accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('第' + str(epoch) + '次大循环，测试集准确率为' + str(accuracy_rate))
    # 保存
    saver.save(sess,'net/my_net.ckpt')    
    
# tf.logging.set_verbosity(old_v)   

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
第0次大循环，测试集准确率为0.8107
第10次大循环，测试集准确率为0.9094
第20次大循环，测试集准确率为0.9166
第30次大循环，测试集准确率为0.9202
第40次大循环，测试集准确率为0.9216
第50次大循环，测试集准确率为0.9231


In [5]:
# 相当于对比 没有使用模型 和 使用了前面保存了的模型 的正确率
with tf.Session() as sess:
    sess.run(init)
    print('瞎猜'+ str(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})))
    #载入
    saver.restore(sess,'net/my_net.ckpt')
    print('使用模型'+ str(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})))

瞎猜0.098
使用模型0.9231
